In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

torch.manual_seed(2019)

In [2]:
trans = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = torchvision.datasets.MNIST('./data', train=True, transform = trans, download=True)
test_dataset = torchvision.datasets.MNIST('./data', train=False, transform = trans, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
import torch.nn as nn
import torch.nn.functional as F
class mNet(nn.Module):
    def __init__(self):
        super(mNet,self).__init__()
        self.conv1=nn.Conv2d(1, 200, 5)
        self.conv2=nn.Conv2d(200, 200, 5)
        self.maxpool=nn.MaxPool2d(2,2)
        self.dropout=nn.Dropout2d()
        self.fc1=nn.Linear(200*4*4,200)
        self.fc2=nn.Linear(200,10)
        
    def forward(self, x):
        x=F.relu(self.maxpool(self.conv1(x)))
        x=F.relu(self.maxpool(self.dropout(self.conv2(x))))
        x=x.view(-1,200*4*4)
        x=F.relu(self.fc1(x))
        x=F.dropout(x, training=self.training)
        x=self.fc2(x)
        return F.log_softmax(x)

In [4]:
import torch.optim as optim
model = mNet().cuda()
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
from torch.autograd import Variable
def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    else:
        model.eval()
        volatile=True
        
    running_correct = 0
    running_loss = 0.0
    for i, (data,target) in enumerate(data_loader):
        if phase == 'training':
            opt.zero_grad()
        data,target = Variable(data,volatile).cuda(),Variable(target).cuda()

        output=model(data)

        loss=F.nll_loss(output,target)
        running_loss+=F.nll_loss(output, target, size_average=False).item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            opt.step()
            
    loss=1.*running_loss/len(data_loader.dataset)
    accuracy=running_correct.type(torch.FloatTensor)/len(data_loader.dataset)*100.    
    
    print(f'epoch#{epoch}, {phase} loss is {loss:{7}.{3}} and {phase} accuracy is {accuracy:{10}.{4}}')
    return loss,accuracy

In [6]:
for epoch in range(10):
    fit(epoch,model,train_loader,'training')
    fit(epoch,model,test_loader,'validation')
    print()

C:\Users\BK\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\BK\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch#0, training loss is   0.493 and training accuracy is      84.86
epoch#0, validation loss is   0.118 and validation accuracy is      96.56



KeyboardInterrupt: 

In [9]:
from torchvision import models
vgg = models.vgg16(pretrained = True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\BK/.cache\torch\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:51<00:00, 10.8MB/s] 


In [26]:
vgg.features[0].in_channels = 1

In [67]:
for epoch in range(10):
    fit(epoch,vgg,train_loader,'training')
    fit(epoch,vgg,test_loader,'validation')
    print()

RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small

In [66]:
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [85]:
vgg.features

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [110]:
m=nn.MaxPool2d(2,2,1,2)
x=torch.randn(1,64,8,8)
r=m(x)
print(x[0][0])
print(r[0][0])

tensor([[ 0.4999,  1.6307, -0.6516, -1.6171,  1.1146,  0.2971, -0.3186,  0.7052],
        [ 0.2684, -0.4250,  0.2890,  1.8758,  0.2710, -0.5698, -0.9614,  0.6702],
        [-1.4438, -0.6670, -0.4135, -0.3205, -2.0053,  0.0420,  0.4941,  3.2793],
        [-0.5916,  0.1412, -1.8025, -0.5303, -1.1449,  0.6675, -0.1135,  2.0366],
        [-1.1058, -0.5507,  0.1640,  0.6122, -0.6212, -0.6572,  0.9207, -0.1635],
        [-1.7463, -0.5174, -1.5851,  0.1939,  0.2564, -1.2144, -0.4731,  1.7697],
        [-0.0811,  0.5336, -1.1154, -2.8417,  1.2070,  0.1640, -0.0972,  1.0296],
        [-0.7371,  0.2543, -0.0179, -0.2472,  1.1985,  0.6002,  0.0383,  0.0512]])
tensor([[-0.4250,  1.8758,  1.8758,  0.6702],
        [ 0.1412,  1.8758,  1.8758,  2.0366],
        [ 0.1412,  0.1939,  0.6675,  2.0366],
        [ 0.2543,  0.2543,  0.6002,  1.7697]])


In [107]:
m

MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)